**1.Download the dataset**

**2.Import required library**

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras_preprocessing import sequence
from keras.utils import to_categorical
from keras.models import load_model

**3.Read Dataset** 

In [2]:
data = pd.read_csv('spam.csv',delimiter=',',encoding='latin-1')
data.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [3]:
data.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'],axis=1,inplace=True) #dropping unwanted columns
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   v1      5572 non-null   object
 1   v2      5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [5]:
data.groupby(['v1']).size()

v1
ham     4825
spam     747
dtype: int64

In [6]:
X = data.v2
Y = data.v1
le = LabelEncoder()
Y = le.fit_transform(Y)
Y = Y.reshape(-1,1)

In [7]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.15)

In [8]:
max_words = 1000
max_len = 150
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)

sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

**4.Create Model and 5. Add Layers (LSTM, Dense-(Hidden Layers), Output)**

In [9]:
inputs = Input(name='InputLayer',shape=[max_len])
layer = Embedding(max_words,50,input_length=max_len)(inputs)
layer = LSTM(64)(layer)
layer = Dense(256,name='FullyConnectedLayer1')(layer)
layer = Activation('relu')(layer)
layer = Dropout(0.5)(layer)
layer = Dense(1,name='OutputLayer')(layer)
layer = Activation('sigmoid')(layer)

**6.Compile the model**

In [10]:
model = Model(inputs=inputs,outputs=layer)
model.summary()
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 InputLayer (InputLayer)     [(None, 150)]             0         
                                                                 
 embedding (Embedding)       (None, 150, 50)           50000     
                                                                 
 lstm (LSTM)                 (None, 64)                29440     
                                                                 
 FullyConnectedLayer1 (Dense  (None, 256)              16640     
 )                                                               
                                                                 
 activation (Activation)     (None, 256)               0         
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                             

**7.Fit the Model**

In [11]:
model.fit(sequences_matrix,Y_train,batch_size=128,epochs=10,
          validation_split=0.2)

Epoch 1/10
30/30 [==============================] - 15s 284ms/step - loss: 0.3044 - accuracy: 0.8865 - val_loss: 0.1475 - val_accuracy: 0.9494
Epoch 2/10
30/30 [==============================] - 8s 264ms/step - loss: 0.0852 - accuracy: 0.9802 - val_loss: 0.0639 - val_accuracy: 0.9821
Epoch 3/10
30/30 [==============================] - 8s 262ms/step - loss: 0.0458 - accuracy: 0.9876 - val_loss: 0.0639 - val_accuracy: 0.9821
Epoch 4/10
30/30 [==============================] - 8s 264ms/step - loss: 0.0378 - accuracy: 0.9894 - val_loss: 0.0515 - val_accuracy: 0.9842
Epoch 5/10
30/30 [==============================] - 8s 265ms/step - loss: 0.0307 - accuracy: 0.9902 - val_loss: 0.0541 - val_accuracy: 0.9831
Epoch 6/10
30/30 [==============================] - 8s 264ms/step - loss: 0.0217 - accuracy: 0.9937 - val_loss: 0.0581 - val_accuracy: 0.9831
Epoch 7/10
30/30 [==============================] - 8s 266ms/step - loss: 0.0184 - accuracy: 0.9947 - val_loss: 0.0571 - val_accuracy: 0.9842
Epoch

**8.Save the Model**

In [12]:
model.save("model.h1")

**9.Test the mode**l

In [13]:
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix  = sequence.pad_sequences(test_sequences,maxlen=max_len)

In [14]:
accuracy = model.evaluate(test_sequences_matrix,Y_test)
print('Accuracy: {:0.3f}'.format(accuracy[1]))

27/27 [==============================] - 1s 23ms/step - loss: 0.0610 - accuracy: 0.9880
Accuracy: 0.988


In [15]:
y_pred = model.predict(test_sequences_matrix)
print(y_pred[25:40].round(3))

27/27 [==============================] - 2s 28ms/step
[[1.   ]
 [1.   ]
 [0.   ]
 [0.   ]
 [0.   ]
 [0.   ]
 [0.   ]
 [0.   ]
 [0.001]
 [0.   ]
 [0.   ]
 [0.   ]
 [0.   ]
 [0.   ]
 [0.001]]


In [16]:
print(Y_test[25:40])

[[1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]]
